In [38]:
# Need only first time
#conda install -c conda-forge python-dotenv
#conda install -c conda-forge pydot

In [135]:
%matplotlib inline

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import datetime
import librosa
import librosa.display
  
import sys
import pathlib
sys.path.insert(1, './fma_metadata/')
import utils

plt.rcParams['figure.figsize'] = (17, 5)

In [147]:
# Load metadata and features.
genres = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/genres.csv')
features = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/features.csv')
echonest = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/echonest.csv')
tracks = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/tracks.csv')


np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape

((106574, 52), (163, 4), (106574, 518), (13129, 249))

# ** Tracks **

In [122]:
# checks percentage of available data
print('Available data - track subset - Tracks.csv (%)')
((abs(tracks['track'].isnull().sum()-tracks.shape[0])/tracks.shape[0])*100).round(2).sort_values(ascending=False)

Available data - track subset - Tracks.csv (%)


bit_rate         100.00
genres           100.00
tags             100.00
number           100.00
listens          100.00
interest         100.00
comments         100.00
genres_all       100.00
title            100.00
favorites        100.00
duration         100.00
date_created     100.00
license           99.92
genre_top         46.54
language_code     14.10
date_recorded      5.78
composer           3.44
information        2.20
publisher          1.19
lyricist           0.29
dtype: float64

In [123]:
print('Available data - album subset - Tracks.csv (%)')
((abs(tracks['album'].isnull().sum()-tracks.shape[0])/tracks.shape[0])*100).round(2).sort_values(ascending=False)

Available data - album subset - Tracks.csv (%)


comments         100.00
favorites        100.00
id               100.00
listens          100.00
tags             100.00
tracks           100.00
title             99.04
date_created      96.69
type              93.89
information       78.02
date_released     65.96
producer          16.95
engineer          14.35
dtype: float64

In [124]:
print('Available data - artist subset - Tracks.csv (%)')
((abs(tracks['artist'].isnull().sum()-tracks.shape[0])/tracks.shape[0])*100).round(2).sort_values(ascending=False)

Available data - artist subset - Tracks.csv (%)


name                 100.00
comments             100.00
favorites            100.00
id                   100.00
tags                 100.00
date_created          99.20
website               74.37
bio                   66.77
location              65.88
members               43.96
latitude              41.80
longitude             41.80
active_year_begin     21.31
associated_labels     13.39
related_projects      12.34
wikipedia_page         5.24
active_year_end        5.04
dtype: float64

In [127]:
tracks['track'].describe()

,bit_rate,comments,duration,favorites,interest,listens,number
count,106574.000000,106574.000000,106574.000000,106574.000000,1.065740e+05,106574.000000,106574.000000
mean,263274.695048,0.031621,277.849100,3.182521,3.541310e+03,2329.353548,8.260945
std,67623.443584,0.321993,305.518553,13.513820,1.901743e+04,8028.070647,15.243271
min,-1.000000,0.000000,0.000000,0.000000,2.000000e+00,0.000000,0.000000
25%,192000.000000,0.000000,149.000000,0.000000,5.990000e+02,292.000000,2.000000
50%,299914.000000,0.000000,216.000000,1.000000,1.314000e+03,764.000000,5.000000
75%,320000.000000,0.000000,305.000000,3.000000,3.059000e+03,2018.000000,9.000000
max,448000.000000,37.000000,18350.000000,1482.000000,3.293557e+06,543252.000000,255.000000


In [133]:
tracks['track']

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155316,320000,0,NaN,2017-03-30 15:23:34,NaT,162,1,Rock,[25],"[25, 12]",NaN,122,NaN,Creative Commons Attribution-NonCommercial-NoD...,102,NaN,3,NaN,[],The Auger
155317,320000,0,NaN,2017-03-30 15:23:36,NaT,217,1,Rock,[25],"[25, 12]",NaN,194,NaN,Creative Commons Attribution-NonCommercial-NoD...,165,NaN,4,NaN,[],Let's Skin Ruby
155318,320000,0,NaN,2017-03-30 15:23:37,NaT,404,2,Rock,[25],"[25, 12]",NaN,214,NaN,Creative Commons Attribution-NonCommercial-NoD...,168,NaN,6,NaN,[],My House Smells Like Kim Deal/Pulp


In [128]:
tracks['album'].describe()

,comments,favorites,id,listens,tracks
count,106574.000000,106574.000000,106574.000000,1.065740e+05,106574.000000
mean,0.394946,1.286927,12826.933914,3.212031e+04,19.721452
std,2.268915,3.133035,6290.261805,1.478532e+05,39.943673
min,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000
25%,0.000000,0.000000,7793.000000,3.361000e+03,7.000000
50%,0.000000,0.000000,13374.000000,8.982000e+03,11.000000
75%,0.000000,1.000000,18203.000000,2.363500e+04,17.000000
max,53.000000,61.000000,22940.000000,3.564243e+06,652.000000


In [129]:
tracks['artist'].describe()

,comments,favorites,id,latitude,longitude
count,106574.000000,106574.000000,106574.000000,44544.000000,44544.000000
mean,1.894702,30.041915,12036.770404,39.901626,-38.668642
std,6.297679,100.511408,6881.420867,18.240860,65.237220
min,-1.000000,-1.000000,1.000000,-45.878760,-157.526855
25%,0.000000,1.000000,6443.000000,39.271398,-79.997459
50%,0.000000,5.000000,12029.500000,41.387917,-73.554431
75%,1.000000,16.000000,18011.000000,48.856930,4.351710
max,79.000000,963.000000,24357.000000,67.286005,175.277000


In [126]:
# display partitions of tracks.csv
ipd.display(tracks['track'].head())
ipd.display(tracks['album'].head())
ipd.display(tracks['artist'].head())
ipd.display(tracks['set'].head())

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type
track_id,,,,,,,,,,,,,
2,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
3,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
5,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
10,0,2008-11-26 01:45:08,2008-02-06,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album
20,0,2008-11-26 01:45:05,2009-01-06,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album


,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page
track_id,,,,,,,,,,,,,,,,,
2,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
3,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
5,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
10,NaT,NaT,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"[philly, kurt vile]",http://kurtvile.com,NaN
20,1990-01-01,2011-01-01,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"[instrumentals, experimental pop, post punk, e...",NaN,NaN


,split,subset
track_id,,
2,training,small
3,training,medium
5,training,small
10,training,small
20,training,large


##  Data cleaning & preparation - tracks.csv
1. all unnecessary rows were removed (after they've been carefully analysed)
2. <b>tracks_df</b> is the base dataset for tracks

In [148]:
# extract year from date_created --> to new col: year_created in track
tracks['track','year_created']= pd.DatetimeIndex(tracks['track','date_created']).year
# remove col date_created
tracks.drop(('track','date_created'),axis=1, inplace=True)
#tracks['track'].head()

In [156]:
tracks['track'].columns

Index(['bit_rate', 'comments', 'composer', 'date_recorded', 'duration',
       'favorites', 'genre_top', 'genres', 'genres_all', 'information',
       'interest', 'language_code', 'license', 'listens', 'lyricist', 'number',
       'publisher', 'tags', 'title', 'year_created'],
      dtype='object')

In [157]:
tracks['artist'].columns

Index(['active_year_begin', 'active_year_end', 'associated_labels', 'bio',
       'comments', 'date_created', 'favorites', 'id', 'latitude', 'location',
       'longitude', 'members', 'name', 'related_projects', 'tags', 'website',
       'wikipedia_page'],
      dtype='object')

In [161]:
tracks['album'].columns

Index(['comments', 'date_created', 'date_released', 'engineer', 'favorites',
       'id', 'information', 'listens', 'producer', 'tags', 'title', 'tracks',
       'type'],
      dtype='object')

In [255]:
# generating new dataset with filtered columns
tracks_df = tracks[
    [('artist','name'),('track','title'),('track','year_created'),('track','duration'),
     ('track','listens'),('track','favorites'),('track','bit_rate'),('track','interest'),('track','license'),
     ('track','genre_top'),('track','genres_all'),('album','title'),('album','tracks'),('album','listens'),
    ('set','split'),('set','subset')]
      ]
# display final dataset 
tracks_df.head()

artist            track                                          \
                name            title year_created duration listens favorites   
track_id                                                                        
2               AWOL             Food         2008      168    1293         2   
3               AWOL     Electric Ave         2008      237     514         1   
5               AWOL       This World         2008      206    1151         6   
10         Kurt Vile          Freeway         2008      161   50135       178   
20        Nicky Cook  Spiritual Level         2008      311     361         0   

                                                                               \
         bit_rate interest                                            license   
track_id                                                                        
2          256000     4656  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
3          256000     1470  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
5          256000     1933  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
10         192000    54881  Attribution-NonCommercial-NoDerivatives (aka M...   
20         256000      978  Attribution-NonCommercial-NoDerivatives (aka M...   

                                                      album                 \
         genre_top         genres_all                 title tracks listens   
track_id                                                                     
2          Hip-Hop               [21]  AWOL - A Way Of Life      7    6073   
3          Hip-Hop               [21]  AWOL - A Way Of Life      7    6073   
5          Hip-Hop               [21]  AWOL - A Way Of Life      7    6073   
10             Pop               [10]     Constant Hitmaker      2   47632   
20             NaN  [17, 10, 76, 103]                 Niris     13    2710   

               set          
             split  subset  
track_id                    
2         training   small  
3         training  medium  
5         training   small  
10        training   small  
20        training   large

In [195]:
# check missing values in tracks_df.csv
tracks_df.isnull().sum()

artist  name                0
track   title               1
        year_created        0
        duration            0
        listens             0
        favorites           0
        bit_rate            0
        interest            0
        license            87
        genre_top       56976
        genres_all          0
album   title            1025
        tracks              0
        listens             0
dtype: int64

In [219]:
# drop rows with missing title (album) as it is not possible to retreive it 
tracks_df.drop(index = tracks_df[tracks_df['album','title'].isnull()].index, inplace=True)
# drop single row with missing values in title (track)
tracks_df.drop(index = tracks_df[tracks_df['track','title'].isnull()].index, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# <span style="color:red">TODO:</span>  REPLACE MISSING VALUES IN GENRES_ALL AND DECIDE IF IT'S WORTH IT TO KEEP LICENSE 

# ** Features **

<b>UNDERSTANDING FEATURES FOR SIGNAL RECOGNITION:</b>

1. <b>zcr</b>:  ZERO-CROSSING-RATE is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive --> useful for classifying percussive sounds.

2. <b>tonnetz</b>:  is a conceptual lattice diagram representing tonal space; Tonnetz can be used to show traditional harmonic relationships in European classical music;

3. <b>chroma_cens</b>:  relates to the twelve different pitch classes; are a powerful tool for analyzing music whose pitches can be meaningfully categorized --> MAIN CHARACT: they capture harmonic and melodic characteristics of music, while being robust to changes in timbre and instrumentation;

4. <b>spectral_bandwidth</b>: is the wavelength interval over which the magnitude of all spectral components is equal to or greater than a specified fraction of the magnitude of the component having the maximum value;

5.  <b>spectral_centroid</b>: is a measure used in digital signal processing to characterise a spectrum. It indicates where the center of mass of the spectrum is located. Perceptually, it has a robust connection with the impression of brightness of a sound;

6. <b>spectral roll-off</b>: The roll-off frequency is defined as the frequency under which some percentage (cutoff) of the total energy of the spectrum is contained. The roll-off frequency can be used to distinguish between harmonic (below roll-off) and noisy sounds (above roll-off);

7. <b>spectral_contrast </b>: is defined as the decibel difference between peaks and valleys in the spectrum.

In [7]:
print('{1} features for {0} tracks'.format(*features.shape))
columns = ['mfcc', 'chroma_cens', 'tonnetz', 'spectral_contrast']
columns.append(['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff'])
columns.append(['rmse', 'zcr'])
for column in columns:
    ipd.display(features[column].head().style.format('{:.2f}'))

518 features for 106574 tracks


In [17]:
trackrep = ProfileReport(tracks['track'])
trackrep.to_file(output_file='profiling/tracks/track-profiling.html')
trackrep = ProfileReport(tracks['album'])
trackrep.to_file(output_file='profiling/tracks/album-profiling.html')
trackrep = ProfileReport(tracks['artist'])
trackrep.to_file(output_file='profiling/tracks/artist-profiling.html')
trackrep = ProfileReport(tracks['set'])
trackrep.to_file(output_file='profiling/tracks/set-profiling.html')

In [11]:
genrep = ProfileReport(genres)
genrep.to_file(output_file='profiling/genres-profiling.html')

In [31]:
echorep = ProfileReport(echonest['echonest']['audio_features'])
echorep.to_file(output_file='profiling/echonest/audio_features.html')

echorep = ProfileReport(echonest['echonest']['metadata'])
echorep.to_file(output_file='profiling/echonest/metadata.html')

echorep = ProfileReport(echonest['echonest']['ranks'])
echorep.to_file(output_file='profiling/echonest/ranks.html')

echorep = ProfileReport(echonest['echonest']['social_features'])
echorep.to_file(output_file='profiling/echonest/social_features.html')

echorep = ProfileReport(echonest['echonest']['temporal_features'])
echorep.to_file(output_file='profiling/echonest/temporal_features.html')

KeyboardInterrupt: 

In [22]:
#featrep = ProfileReport(features)
#featrep.to_file(output_file='profiling/features-profiling.html')
#for col in features.columns: 
#    print(col)